# Einführung in die Zeitreihenanalyse

Beispiel: Möglicher Einfluss des Klimawandels auf Temperaturen und Niederschläge in Luzern

_CAS Data Science FHNW,  2022-2024, S. Billeter_

**Hier liegt der Fokus auf den Lösungen der Aufgaben des Skripts *1-Zeitreihen-mit-Übungen.ipynb***

In [ ]:
# Grundlegende Libraries importieren
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt

### Schritt 1: Exploration und technische Vorbereitung

In [ ]:
# Zeitreihendaten einlesen
df_Luzern=pd.read_csv('Luzern-TS.csv',sep=';')

In [ ]:
# Daten erzeugen
df_Luzern['Date']=pd.to_datetime(df_Luzern['Year'].astype(str).str.cat(df_Luzern['Month'].astype(str),sep='-'))
# Die Zeitreihe hat einen Index gesetzt
ts_Luzern=df_Luzern.set_index('Date')

In [ ]:
# Zeitreihe der Temperaturen ab 1900 einpacken
Luzern_Temp=df_Luzern[['Date','Temperature']]
Luzern_Temp=Luzern_Temp.set_index('Date')
Luzern_Temp=Luzern_Temp[Luzern_Temp.index>='1900-01-01']

**Aufgabe:**
Packen Sie die Niederschläge in eine Zeitreihe ein

In [ ]:
# Lösung:
Luzern_Prec=df_Luzern[['Date','Precipitation']]
Luzern_Prec=Luzern_Prec.set_index('Date')
Luzern_Prec=Luzern_Prec[Luzern_Prec.index>='1900-01-01']
Luzern_Prec

### Schritt 2: Modellierung: Zerlegung

In [ ]:
# Libraries für Zeitreihenanalyse
import statsmodels
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf, ccf
from statsmodels.tsa.stattools import adfuller

In [ ]:
# Zerlegung in Komponenten
Luzern_Temp_Komp=seasonal_decompose(Luzern_Temp, model='additive')

In [ ]:
# Die NaNs werden nun entfernt, hier nur am Rand
Temp_Trend=Luzern_Temp_Komp.trend
Temp_Trend=Temp_Trend[Temp_Trend.notnull()]
Temp_Resid=Luzern_Temp_Komp.resid
Temp_Resid=Temp_Resid[Temp_Resid.notnull()]
Temp_Seasonal=Luzern_Temp_Komp.seasonal
Temp_Seasonal=Temp_Seasonal[Temp_Seasonal.notnull()]

**Aufgabe:**
Zerlegen Sie die Niederschläge in Komponenten. Welche Komponenten könnten stationär sein?

In [ ]:
# Lösung
# 1. Zerlegung
Luzern_Nied_Komp=seasonal_decompose(Luzern_Prec, model='additive')
fig=Luzern_Nied_Komp.plot()
fig.set_size_inches(12,10)
fig.tight_layout()

# 2. Entfernung der NaNs
Nied_Trend=Luzern_Nied_Komp.trend
Nied_Trend=Nied_Trend[Nied_Trend.notnull()]
Nied_Resid=Luzern_Nied_Komp.resid
Nied_Resid=Nied_Resid[Nied_Resid.notnull()]
Nied_Seasonal=Luzern_Nied_Komp.seasonal
Nied_Seasonal=Nied_Seasonal[Nied_Seasonal.notnull()]
# print(Nied_Resid)

# 3. Stationaritätstests - Plots s. Zerlegung
r=adfuller(Luzern_Prec,maxlag=12)
print(f'p-Wert ganze Zeitreihe: {r[1]:.6f}')
r=adfuller(Nied_Resid,maxlag=12)
print(f'p-Wert Residuen: {r[1]:.6f}')
r=adfuller(Nied_Trend,maxlag=12)
print(f'p-Wert Trend: {r[1]:.6f}')

### Schritt 2a: Manuelle Zerlegung

In [ ]:
# Weitere Libraries für linearen Trend
from sklearn.linear_model import LinearRegression

**Langfrist-Trend**

Einfaches lineares Modell

In [ ]:
Luzern_Temp_Man=Luzern_Temp
Luzern_Temp_Man['Year']=Luzern_Temp_Man.index.year+Luzern_Temp.index.month/12
# Variablen zur Regression
x=pd.to_numeric(Luzern_Temp_Man['Year']).values.reshape(-1,1)
y=Luzern_Temp_Man['Temperature'].values

In [ ]:
# Fit des linearen Trends
Temp_Linear=LinearRegression().fit(x,y)

In [ ]:
# Residuen (immer noch mit Saisonalität drin)
Luzern_Temp_Man['Temp_Linear']=Temp_Linear.predict(x)
Luzern_Temp_Man['Resid']=Luzern_Temp_Man['Temperature']-Luzern_Temp_Man['Temp_Linear']

**Quadratischen Trend auch entfernen**

*Aufpassen mit der Verhersagekraft*

In [ ]:
# Quadratischen Trend herausnehmen - Variablen vorbereiten
Luzern_Temp_Quad=Luzern_Temp
Luzern_Temp_Quad['Year']=Luzern_Temp_Quad.index.year+Luzern_Temp_Quad.index.month/12
# Variablen zur Regression
Luzern_Temp_Quad['Year1']=Luzern_Temp_Quad['Year']-2000
Luzern_Temp_Quad['Year2']=np.square(Luzern_Temp_Quad['Year']-2000)

In [ ]:
# Zeit nach 1950
Luzern_Temp_Quad=Luzern_Temp_Quad[Luzern_Temp_Quad['Year']>1950]

### Schritt 3: Analyse

**Aufgabe:**
Analysieren Sie die Autokorrelationsstruktur der Niederschläge. Was lässt sich daraus ablesen?

In [ ]:
# Lösung
# 1. Residuen analysieren
print('Residuen')
fig,axs=plt.subplots(1,3)
fig.set_size_inches(16,5)
Nied_Resid.plot(title='Niederschlag Luzern, Residuen',ax=axs[0])
plot_acf(Temp_Resid,title='ACF Niederschlag Luzern, Residuen',ax=axs[1])
plot_pacf(Temp_Resid,title='PACF Niederschlag Luzern, Residuen',ax=axs[2])
plt.show()

# 2. Beitrag der Komponenten
print('Vergleich mit Komponenten')
fig,axs=plt.subplots(3,3)
fig.set_size_inches(16,16)
Nied_Seasonal.plot(title='Niederschlag Luzern, Saison',ax=axs[0,0])
plot_acf(Nied_Seasonal,title='ACF Niederschlag Luzern, Saison',ax=axs[0,1])
plot_pacf(Nied_Seasonal,title='PACF Niederschlag Luzern, Saison',ax=axs[0,2])
Nied_Trend.plot(title='Niederschlag Luzern, Trend',ax=axs[1,0])
plot_acf(Nied_Trend,title='ACF Niederschlag Luzern, Trend',ax=axs[1,1])
plot_pacf(Nied_Trend,title='PACF Niederschlag Luzern, Trend',ax=axs[1,2])
Luzern_Prec.plot(title='Niederschlag Luzern gesamt',ax=axs[2,0])
plot_acf(Luzern_Prec,title='ACF Niederschlag Luzern gesamt',ax=axs[2,1])
plot_pacf(Luzern_Prec,title='PACF Niederschlag Luzern gesamt',ax=axs[2,2])
plt.show()

**Welcher Zusammenhang besteht zwischen Temperatur und Niederschlag?**

In [ ]:
# Kreuzkorrelationsfunktion
fig,ax=plt.subplots(figsize=(12,6))
ax.stem(ccf(Temp_Resid,Nied_Resid)[0:24])
ax.set_title('Korrelation Temperatur und Niederschlag')
ax.set_xlabel('Lag')
ax.set_ylabel('CCF')
plt.show()

In [ ]:
# Welches Signal ist zuerst - anhand von Demodaten
fig,[ax1,ax2]=plt.subplots(1,2,figsize=(16,6))
marketing = np.array([3, 4, 5, 5, 7, 9, 13, 15, 12, 10, 8, 8])
revenue = np.array([21, 19, 22, 24, 25, 29, 30, 34, 37, 40, 35, 30])
print(ccf(marketing, revenue, adjusted=False))
print(ccf(revenue, marketing, adjusted=False))
ax1.plot(marketing,color='blue')
ax1.plot(revenue,color='red')
ax1.set_title('Zeitreihen')
ax2.plot(ccf(revenue,marketing)[0:10],color='black')
ax2.plot(ccf(marketing,revenue)[0:10],color='grey')
ax2.set_title('Kreuzkorrelation (Schwarz: blau-rot)')
plt.show()

In [ ]:
# Kreuzkorrelationsfunktion - zeitlicher Zusammenhang
fig,[ax1,ax2]=plt.subplots(1,2,figsize=(16,6))
ax1.stem(ccf(Temp_Resid,Nied_Resid)[0:24])
ax1.set_title('Temperatur mit Niederschlag')
ax2.stem(ccf(Nied_Resid,Temp_Resid)[0:24])
ax2.set_title('Niederschlag mit Temperatur')
plt.show()

**Aufgabe:** Was können Sie aus diesen Kreuzkorrelationen herauslesen?

In [ ]:
# Lösung:
# - In einem niederschlagsreichen Monat ist es deutlich kühler als im Durchschnitt erwartet
# - Einem niederschlagsreichen Monat kann ein kühlerer Monat vorangehen (was könnte hier die Autokorrelation aussagen?)
# - Einem überdurchschnittlich warmen Monat kann ein überdurchschnittlich niederschlagsreicher Monat vorangehen
#
# Folgeaufgabe: Warum sind die Lags um +/-6 wenig aussagekräftig?

**Die statistischen Korrelationen sagen nicht alles aus**

_Brauchen Sie Ihr Vorwissen_

### Schritt 4: Stochastische Modellierung

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

**Aufgaben:**
1. Finden Sie ein geeignetes ARIMA-Modell der Niederschläge
2. Interpretieren Sie das Ergebnis
3. Schlagen Sie Massnahmen vor, um das Ergebnis aussagekräftiger zu machen

In [ ]:
# Lösung:
# 1a. Korrelogramme
fig,axs=plt.subplots(1,3)
fig.set_size_inches(16,6)
Nied_Resid.plot(title='Niederschlag Luzern, Residuen',ax=axs[0])
plot_acf(Nied_Resid,title='ACF Niederschlag Luzern, Residuen',ax=axs[1])
plot_pacf(Nied_Resid,title='PACF Niederschlag Luzern, Residuen',ax=axs[2])
plt.show()

# 1b. Moving-Average (ARIMA-)Modell
Nied_Modell=ARIMA(Nied_Resid,order=(0,0,5),freq=Nied_Resid.index.inferred_freq)
Nied_resid_fit=Nied_Modell.fit()
print(Nied_resid_fit.summary())

# 2. Interpretationen:
# - Ein noch zu suchender Prozess beeinflusst und stabilisiert die Niederschläge ungefähr innert Monatsfrist
# - Wir sehen Reste noch nicht vollständig entfernter Saisonalitäten und Trends

# 3. 
# - Kürzere Zeitabschnitte
# - Realistischeres Modell der Saisonalitäten
# - Händische Entfernung eines Langfrist-Trends
# - Unterschiedliche Zeitskalen
# - Vorwissen über den verborgenen Prozess verwenden
# - Deterministische Komponenten tiefer analysieren

**Anwendungen**
_Ausserhalb dieses Skripts (kein Standardvorgehen)_

*Folien 35-37*

- Vorhersagen
- Bedingte Vorhersagen
- Weiterverwendung in anderen Modellen
- Extremwertanalysen


### Vorhersagen

Hier kommt alles zusammen

*Einige Beispiele, es müssen unbedingt an die Problemstellungen angepasst werden*

In [ ]:
Luzern_Temp_Man['Seasonal']=Temp_Seasonal
Luzern_Temp_Man['Resid_Seasonal']=Luzern_Temp_Man['Temperature']-Luzern_Temp_Man['Seasonal']-Luzern_Temp_Man['Temp_Linear']
Luzern_Temp_Man_Modell=ARIMA(Luzern_Temp_Man['Resid_Seasonal'],order=(1,0,0),freq=Luzern_Temp_Man.index.inferred_freq).fit()

**Aufgaben:**

1. Am 31. Mai 2030 sei es 20.1 Grad C. Welche Temperatur ist am 1. Juni 2030 zu erwarten?
2. Was sagen die Residuen der Jahre 2000 bis 2021 über das Modell aus? Welche Komponente muss vor allem verbessert werden?

In [ ]:
# Lösungen:
# 1. Aufgabe: Residuum des Vormonats, dann AR(1)-Simulation
t=[[2030+(31+28+31+30+31)/365.24]]
t_1=[[2030+(31+28+31+30+3)/365.24]]
T_t=Temp_Linear.predict(t)[0]
S_t=Luzern_Temp_Man['Seasonal']['2021-06-01']
R_t_1=20.1-Temp_Linear.predict(t_1)[0]-Luzern_Temp_Man['Seasonal']['2021-05-01']
R_t=Luzern_Temp_Man_Modell.params['ar.L1']*R_t_1 # Warum mit w_t=0?
x_t=T_t+S_t+R_t
print(f"Temperatur: {x_t:.3f}, Residuum (t): {R_t:.3f}, Residuum (t-1): {R_t_1:.3f} Grad C")
# 2. Aufgabe: Wird besprochen. Erster Schritt: Werte überprüfen
ZR=Luzern_Temp_Man['Resid_Seasonal'][Luzern_Temp_Man.index>='2000-01-01']
plt.plot(ZR)
plt.show()
print(f"Mittelwert: {ZR.mean():.3f}, Standardabweichung: {np.sqrt(ZR.var()):.3f}")